See if we can extract a FITS image and apply some external code to it. 

In [1]:
from lsst.daf.butler import Butler
from lsst.daf.butler.registry import Registry


In [2]:
config = '/repo/dc2/'
collection = '2.2i/runs/DP0.1'
butler = Butler(config, collections=collection)

registry = butler.registry

Listing the images you want will have to be done with butler
Using one of the tutorial queries.
Here we will get a known visit and all the detectors from that visit. 

In [3]:
datasetType = 'calexp'
dataId = {'visit': 733724}
datasetRefs = set(registry.queryDatasets(datasetType, dataId=dataId))

print(f"Found {len(datasetRefs)} detectors")

Found 189 detectors


Now we have 189  ccd images for one focal plane  we can write out FITS files from Python objects. 

Write out fits files for the first count images. 
By default you will not have enough space to write 189 files.

In [4]:
for count, exp in enumerate(datasetRefs):
    fn=f"Rubin-calexp-{exp.dataId['visit']}-{exp.dataId['detector']}.fits"
    calexp = butler.get('calexp',exp.dataId)
    calexp.writeFits(fn)
    if (count > 20):
        break

print (f"{count+1} exposures output")

22 exposures output


Now we have Fits files we can use with external code.
We get the list of files we made and process them with SEP (Source Extractor Python) as an example of external code. 
We will make a list of Objects and store that.

In [5]:
import glob
filelist = glob.glob('Rubin-calexp*.fits')

['Rubin-calexp-733724-164.fits', 'Rubin-calexp-733724-109.fits', 'Rubin-calexp-733724-115.fits', 'Rubin-calexp-733724-19.fits', 'Rubin-calexp-733724-146.fits', 'Rubin-calexp-733724-59.fits', 'Rubin-calexp-733724-184.fits', 'Rubin-calexp-733724-152.fits', 'Rubin-calexp-733724-127.fits', 'Rubin-calexp-733724-70.fits', 'Rubin-calexp-733724-66.fits', 'Rubin-calexp-733724-81.fits', 'Rubin-calexp-733724-162.fits', 'Rubin-calexp-733724-14.fits', 'Rubin-calexp-733724-172.fits', 'Rubin-calexp-733724-113.fits', 'Rubin-calexp-733724-158.fits', 'Rubin-calexp-733724-15.fits', 'Rubin-calexp-733724-93.fits', 'Rubin-calexp-733724-96.fits', 'Rubin-calexp-733724-105.fits', 'Rubin-calexp-733724-135.fits']


In [7]:
from astropy.io import fits
import sep
import csv
catfile = 'catalog.csv'
outfile = open(catfile,'w')
catalog = csv.writer(outfile,delimiter=',')
ocount = 0
               
for ffile in filelist:
    hdul = fits.open(ffile)  
    data = hdul[1].data.byteswap().newbyteorder()  # sep wants this 
    bkg = sep.Background(data)
    # subtract the background
    data_sub = data - bkg
    objects = sep.extract(data_sub, 1.5, err=bkg.globalrms)
    for o in objects:
        catalog.writerow(o)
    ocount = ocount + len(objects)
outfile.close()
print(f"Extracted {ocount} objects from {len(filelist)} images/ccds written to {catfile}.")

Extracted 30425 objects from 22 images/ccds written to catalog.csv.
